# Advanced Time Series Forecasting — ARIMA vs LSTM

This notebook provides a more industrial comparison between a classical statistical model (**ARIMA**) and a deep learning baseline (**LSTM**) on the same synthetic dataset (trend + seasonality + noise).

Deliverables:
- clear train/val split
- standardized metrics (MAE/RMSE)
- reproducible pipelines

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt

SEED = 1337
rng = np.random.default_rng(SEED)

def make_series(n=1500, season=24, noise=0.6):
    t = np.arange(n)
    trend = 0.0025 * t
    seasonal = 1.2 * np.sin(2 * np.pi * t / season) + 0.4 * np.sin(2 * np.pi * t / (season*7))
    y = 10 + trend + seasonal + rng.normal(0, noise, size=n)
    return y

y = make_series()
split = int(len(y) * 0.8)
y_train, y_val = y[:split], y[split:]
len(y_train), len(y_val)

## ARIMA (statsmodels)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# simple order for demo; in production you'd grid-search / use auto_arima
model = ARIMA(y_train, order=(2, 1, 2))
fit = model.fit()
pred_arima = fit.forecast(steps=len(y_val))

mae_arima = float(np.mean(np.abs(pred_arima - y_val)))
rmse_arima = float(sqrt(np.mean((pred_arima - y_val)**2)))
mae_arima, rmse_arima

## LSTM (PyTorch)

In [ ]:
import torch
import torch.nn as nn

torch.manual_seed(SEED)

def make_windows(arr, window=48):
    X, Y = [], []
    for i in range(len(arr) - window):
        X.append(arr[i:i+window])
        Y.append(arr[i+window])
    return np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)

window = 48
Xtr, Ytr = make_windows(y_train, window=window)
Xva, Yva = make_windows(np.concatenate([y_train[-window:], y_val]), window=window)

# normalize using train stats only
mu, sig = Xtr.mean(), Xtr.std() + 1e-8
Xtr = (Xtr - mu) / sig
Xva = (Xva - mu) / sig
Ytr_n = (Ytr - mu) / sig

Xtr_t = torch.from_numpy(Xtr).unsqueeze(-1)
Ytr_t = torch.from_numpy(Ytr_n).unsqueeze(-1)
Xva_t = torch.from_numpy(Xva).unsqueeze(-1)

class LSTMForecaster(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, dropout=0.15, batch_first=True)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

model = LSTMForecaster()
opt = torch.optim.AdamW(model.parameters(), lr=2e-3)
loss_fn = nn.MSELoss()

# tiny training loop (kept short for notebook); in production use DataLoader + early stopping
model.train()
for epoch in range(8):
    opt.zero_grad()
    pred = model(Xtr_t)
    loss = loss_fn(pred, Ytr_t)
    loss.backward()
    opt.step()
    if epoch % 2 == 0:
        print('epoch', epoch, 'loss', float(loss))

model.eval()
with torch.no_grad():
    p = model(Xva_t).squeeze(-1).numpy()
pred_lstm = p * sig + mu

mae_lstm = float(np.mean(np.abs(pred_lstm - y_val)))
rmse_lstm = float(sqrt(np.mean((pred_lstm - y_val)**2)))
mae_lstm, rmse_lstm

## Summary

In [ ]:
pd.DataFrame({
  'model': ['ARIMA(2,1,2)', 'LSTM'],
  'MAE': [mae_arima, mae_lstm],
  'RMSE': [rmse_arima, rmse_lstm],
}).sort_values('RMSE')